# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [10]:
# Your code here
cur.execute("""SELECT lastName, firstName 
               FROM employees
               JOIN offices 
               USING(officeCode)
               WHERE city = 'Boston';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,lastName,firstName
0,Firrelli,Julie
1,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [48]:
# Your code here
cur.execute("""SELECT offices.officeCode, offices.city, 
               count(employees.employeeNumber) AS totalEmployees
               FROM offices
               LEFT JOIN employees
               USING(officeCode)
               GROUP BY officeCode
               HAVING totalEmployees = 0;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,officeCode,city,totalEmployees
0,27,Boston,0


## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name 
# for each product that each employee has sold

In [24]:
# Your code here
# Select the number of customers each employee has
cur.execute("""SELECT employeeNumber, firstName, lastName, 
               count(customerName) AS numCustomers
               FROM employees
               JOIN customers
               ON employees.employeeNumber = customers.salesRepEmployeeNumber
               GROUP BY employeeNumber
               ORDER BY numCustomers DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,numCustomers
0,1401,Pamela,Castillo,10
1,1504,Barry,Jones,9
2,1323,George,Vanauf,8
3,1501,Larry,Bott,8
4,1286,Foon Yue,Tseng,7
5,1370,Gerard,Hernandez,7
6,1165,Leslie,Jennings,6
7,1166,Leslie,Thompson,6
8,1188,Julie,Firrelli,6
9,1216,Steve,Patterson,6


In [28]:
# Your code here
# List of products in each product line with how many have been ordered
cur.execute("""SELECT productLines.productLine, products.productCode, products.productName,
               sum(orderdetails.quantityOrdered) AS totalOrdered
               FROM productLines
               JOIN products 
               ON productLines.productLine = products.productLine
               JOIN orderdetails
               ON products.productCode = orderdetails.productCode
               GROUP BY products.productCode
               ORDER BY productLines.productLine, products.productCode
               ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,productLine,productCode,productName,totalOrdered
0,Classic Cars,S10_1949,1952 Alpine Renault 1300,961
1,Classic Cars,S10_4757,1972 Alfa Romeo GTA,1030
2,Classic Cars,S10_4962,1962 LanciaA Delta 16V,932
3,Classic Cars,S12_1099,1968 Ford Mustang,933
4,Classic Cars,S12_1108,2001 Ferrari Enzo,1019
...,...,...,...,...
104,Vintage Cars,S24_3816,1940 Ford Delivery Sedan,923
105,Vintage Cars,S24_3969,1936 Mercedes Benz 500k Roadster,824
106,Vintage Cars,S24_4258,1936 Chrysler Airflow,983
107,Vintage Cars,S32_4289,1928 Ford Phaeton Deluxe,972


In [33]:
# Your code here
# How much money has been collected by each office
cur.execute("""SELECT offices.city, sum(payments.amount) AS totalPayments
               FROM offices
               JOIN employees
               ON offices.officeCode = employees.officeCode
               JOIN customers
               ON employees.employeeNumber = customers.salesRepEmployeeNumber
               JOIN payments
               ON customers.customerNumber = payments.customerNumber
               GROUP BY offices.city
               ORDER BY totalPayments DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,city,totalPayments
0,Paris,2819168.90
1,San Francisco,1337439.58
2,London,1324325.90
3,NYC,1072619.47
4,Sydney,1007292.98
5,Boston,835882.33
6,Tokyo,457110.07


## Level Up: Display the names of every individual product that each employee has sold

In [43]:
# Your code here
cur.execute("""SELECT employees.employeeNumber, employees.lastName, employees.firstName,
                products.productName
                FROM employees
                JOIN customers
                ON employees.employeeNumber = customers.salesRepEmployeeNumber
                JOIN orders
                ON customers.customerNumber = orders.customerNumber
                JOIN orderdetails
                ON orders.orderNumber = orderdetails.orderNumber
                JOIN products
                ON orderdetails.productCode = products.productCode
                GROUP BY employees.employeeNumber, products.productName
                ORDER BY employees.employeeNumber, products.productName;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,lastName,firstName,productName
0,1165,Jennings,Leslie,18th Century Vintage Horse Carriage
1,1165,Jennings,Leslie,18th century schooner
2,1165,Jennings,Leslie,1900s Vintage Bi-Plane
3,1165,Jennings,Leslie,1900s Vintage Tri-Plane
4,1165,Jennings,Leslie,1903 Ford Model A
...,...,...,...,...
1363,1702,Gerard,Martin,The Mayflower
1364,1702,Gerard,Martin,The Queen Mary
1365,1702,Gerard,Martin,The Schooner Bluenose
1366,1702,Gerard,Martin,The Titanic


In [53]:
cur.execute("""SELECT firstName, lastName, productName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)
               ORDER BY lastName, firstName, productName;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head(30)

2996


,firstName,lastName,productName
0,Loui,Bondur,18th century schooner
1,Loui,Bondur,1900s Vintage Bi-Plane
2,Loui,Bondur,1900s Vintage Tri-Plane
3,Loui,Bondur,1903 Ford Model A
4,Loui,Bondur,1903 Ford Model A
5,Loui,Bondur,1904 Buick Runabout
6,Loui,Bondur,1911 Ford Town Car
7,Loui,Bondur,1911 Ford Town Car
8,Loui,Bondur,1912 Ford Model T Delivery Wagon
9,Loui,Bondur,1917 Grand Touring Sedan


## Level Up: Display the Number of Products each employee has sold

In [46]:
# Your code here
cur.execute("""SELECT employees.employeeNumber, employees.lastName, employees.firstName,
               sum(orderdetails.quantityOrdered) AS totalProductsSold
               FROM employees
               JOIN customers
               ON employees.employeeNumber = customers.salesRepEmployeeNumber
               JOIN orders
               ON customers.customerNumber = orders.customerNumber
               JOIN orderdetails
               ON orders.orderNumber = orderdetails.orderNumber
               GROUP BY employees.employeeNumber
               ORDER BY totalProductsSold DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,lastName,firstName,totalProductsSold
0,1370,Hernandez,Gerard,14231
1,1165,Jennings,Leslie,11854
2,1401,Castillo,Pamela,9290
3,1501,Bott,Larry,8205
4,1504,Jones,Barry,7486
5,1323,Vanauf,George,7423
6,1612,Marsh,Peter,6632
7,1611,Fixter,Andy,6246
8,1337,Bondur,Loui,6186
9,1216,Patterson,Steve,5561


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!